In [62]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import torch
import pandas as pd
from transformers import AutoTokenizer
from project_name.preprocessing.baseline_preprocessing import BaselinePreprocessor

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=True)

df = pd.read_json("../data/raw/training_merged.json", orient="records", lines=True)
print(df)


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


                                                  tweet emotion  score
0     @xandraaa5 @amayaallyn6 shut up hashtags are c...   anger  0.562
1     it makes me so fucking irate jesus. nobody is ...   anger  0.750
2            Lol Adam the Bull with his fake outrage...   anger  0.417
3     @THATSSHAWTYLO passed away early this morning ...   anger  0.354
4     @Kristiann1125 lol wow i was gonna say really?...   anger  0.438
...                                                 ...     ...    ...
7097  Watch this amazing live.ly broadcast by @kana_...     joy  0.558
7098  Watching @melissamccarthy in #Spy she's one of...     joy  0.780
7099                            Could not be happier!!      joy  0.885
7100  @strictlysimilak something about English spark...     joy  0.360
7101  and I think some of our most spiritually weigh...     joy  0.440

[7102 rows x 3 columns]


In [63]:
label_map = {label: idx for idx, label in enumerate(df["emotion"].unique())}
df["label"] = df["emotion"].map(label_map)

NUM_CLASSES = len(label_map)

print(df.head())


                                               tweet emotion  score  label
0  @xandraaa5 @amayaallyn6 shut up hashtags are c...   anger  0.562      0
1  it makes me so fucking irate jesus. nobody is ...   anger  0.750      0
2         Lol Adam the Bull with his fake outrage...   anger  0.417      0
3  @THATSSHAWTYLO passed away early this morning ...   anger  0.354      0
4  @Kristiann1125 lol wow i was gonna say really?...   anger  0.438      0


In [64]:
import re

def bertweet_preprocess(text):
    text = text.lower()
    text = re.sub(r'@\w+', '@USER', text)
    text = re.sub(r'http\S+', 'HTTPURL', text)
    return text


In [65]:

df["tweet"] = df["tweet"].apply(bertweet_preprocess)

In [66]:
from datasets import Dataset

dataset = Dataset.from_pandas(df[["tweet", "label"]])

In [67]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

def tokenize(example):
    return tokenizer(example["tweet"], truncation=True, padding="max_length")

tokenized_dataset = dataset.map(tokenize, batched=True)
print(tokenized_dataset)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


Map:   0%|          | 0/7102 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dataset({
    features: ['tweet', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7102
})


In [70]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=NUM_CLASSES)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer
)

trainer.train()

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

/Users/Programming/.local/share/virtualenvs/Applied-ML-Template-1-YC0RMAhE/lib/python3.11/site-packages/huggingface_hub/file_download.py:799: UserWarning: Not enough free disk space to download the file. The expected file size is: 542.51 MB. The target location /Users/Programming/.cache/huggingface/hub/models--vinai--bertweet-base/blobs only has 38.53 MB free disk space.
  warnings.warn(


model.safetensors:  33%|###3      | 273M/815M [00:00<?, ?B/s]